In [9]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
import pandas as pd

import time
from dotenv import load_dotenv
import os
from bs4 import BeautifulSoup
import requests
import pprint
max_chrome_driver_path = '/Users/maxlien/Downloads/chromedriver-mac-arm64/chromedriver'

from dotenv import load_dotenv
import os

def config():
    load_dotenv()
  
config()

username = str(os.getenv('MONOPLE_GMAIL'))
password=str(os.getenv('MONOPLE_PASSWORD'))

In [22]:
class StackExchangeScraper:
    def __init__(self, chrome_driver_path=max_chrome_driver_path,username=username, password=password):
        self.driver_path = chrome_driver_path
        self.options = webdriver.ChromeOptions()
        #print(self.options)
        self.service = ChromeService(executable_path=self.driver_path)
        self.driver = webdriver.Chrome(service=self.service) #executable_path= max_chrome_driver_path)#,options=self.options)#executable_path=self.driver_path)
        self.wait = WebDriverWait(self.driver, 10)
        
        self.username =username
        self.password = password

    # ... rest of the class methods remain unchanged ...

# Note: Only the initialization method of the class is shown here for brevity.
# The rest of the class methods would remain unchanged from the previous version.
    def _set_up_agent(self):
        #set useragent
        self.options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

    # def _setup_headless_driver(self):
    #     """
    #     Sets up a headless Selenium Chrome driver.
    #     """
    #     self.options.add_argument('--headless')
    #     self.options.add_argument('--no-sandbox')
    #     self.options.add_argument('--disable-dev-shm-usage')
    #     self.driver = webdriver.Chrome(self.driver_path, options=options)
    def _wait(self,type, name):
        return self.wait.until(
                EC.presence_of_element_located((type, name))
            )
    def _wait_all(self, type, name):
        return self.wait.until(
                EC.visibility_of_all_elements_located((type, name))
            )
    def _login(self, username, password):
        
        
        login_button_XPATH = '/html/body/div[1]/header[1]/div/nav/ol/li[4]/a'
        login_button = self._wait(By.XPATH, login_button_XPATH)
        self.driver.execute_script("arguments[0].scrollIntoView();", login_button)
        login_button.click()


        
    

        username_box = self._wait(By.XPATH, '//*[@id="email"]')
        username_box.send_keys(username)

        password_box = self._wait(By.XPATH,'//*[@id="password"]' )
        password_box.send_keys(password)


        submit_button =self._wait(By.XPATH, '//*[@id="submit-button"]') 
        self.driver.execute_script("arguments[0].scrollIntoView();", submit_button)
        submit_button.click()




    def search_links(self, input):
        self._set_up_agent()
        self.driver.get('https://stackexchange.com')

        self._login(self.username, self.password)

        search_box = self._wait(By.XPATH, '//*[@id="search"]/div/input')
        search_box.send_keys(input)
        search_box.send_keys(Keys.ENTER)

        #warning 
        #potential bug here
        #if been redirected to meta.stackexchange
        #the XPATH might not work
        content = self._wait(By.XPATH, '//*[@id="content"]/div/div[3]')
        html_content = content.get_attribute('outerHTML')
        soup = BeautifulSoup(html_content, 'html.parser')
        links = soup.find_all(class_='result-link')
        # for link in links:
        #     print(link.a['href'])
        urls = [str(link.a['href']) for link in links]
        return urls       
        # try:
        #     #for stack_exchange
        #     #result_link_elements = self._wait_all(By.CSS_SELECTOR, "result-link > a")
        #     #print(result_link_elements)
        #     # Extracting the links within each of the 'result-link' elements
        #     links =[]
        #     for element in self._wait_all(By.CSS_SELECTOR, "result-link > a"):
        #         links.append(element.get_attribute("herf"))
        # except TimeoutException :
        #     #in case it redirect to meta.stackexchange
        #     #result_link_elements = self._wait_all(By.CLASS_NAME, "s-post-summary--content-title > a")
        #     #print(result_link_elements)
        #     # Extracting the links within each of the 'result-link' elements
        #     links = []
        #     for element in self._wait_all(By.CSS_SELECTOR, "s-post-summary--content-title > a"):
        #         links.append(element.get_attribute("innerHTML"))

        
        # return links


    def search_anwser(self,url):
        content = dict()
        answer_body = []


        res = requests.get(str(url))
        soup = BeautifulSoup(res.text, parser='html_parser')
        post_body = soup.find_all('div', class_='s-prose js-post-body',itemprop='text')
        question_title = soup.find('h1',class_='fs-headline1 ow-break-word mb8 flex--item fl1',itemprop='name')

        answer_body = [i.text for i in post_body if i.text]


        content['question_title'] = question_title.text
        content['question'] = answer_body[0]
        content['answer'] = answer_body[1:len(content)]
        content['url'] =f'{url}'

        return content
        



    def scrape(self, input):
        """
        Main method to scrape StackExchange for a given query and return the texts in a list.
        """
        #self._setup_headless_driver()
        
        urls = self.search_links(input)
        response = {}
        for url in urls:
            content = self.search_anwser(url)
            question_title = content['question_title']
            content.pop('question_title')
            response[f'{question_title}'] = content
            

        self.driver.quit()

        return response

# Note: As before, the Selenium portion cannot be executed in this environment. 
# The code should be run in a local environment with the appropriate browser driver.


In [25]:
SS = StackExchangeScraper(chrome_driver_path=max_chrome_driver_path,username=username, password=password)
content =SS.scrape('how to use AWS RDS')
content

{'How to fix an error Plone RelStorage using AWS RDS Read replica by psycopg2': {'question': '\nWe want to use AWS RDS Read replica by psycopg2, but we got an error.\nOur env:\n\nPlone 5\nRelStorage 1.6.3\npsycopg2 2.7.3.2\nAWS RDS PostgreSQL (Read Replica)\nbuildout.conf: rel-storage = read-only true\n\nError:\npsycopg2.NotSupportedError: cannot use serializable mode in a hot standby\nHINT:  You can use REPEATABLE READ instead.\n\nWe set:\ndefault_transaction_isolation  repeatable read\nHow can I do anything?\n',
  'answer': [],
  'url': 'https://stackoverflow.com/questions/47883071/how-to-fix-an-error-plone-relstorage-using-aws-rds-read-replica-by-psycopg2'},
 "How to connect to AWS EC2 and AWS RDS using my company's remote access VPN?": {'question': "\nMy company's AWS security inbound rules are set such that IP's from within company networks can SSH into AWS Resources. However, I want to work from home and be able to SSH into AWS EC2 instances (and also login to RDS instances). \nO

In [26]:
pd.DataFrame.from_dict(content)
pd.to_sql()

,How to fix an error Plone RelStorage using AWS RDS Read replica by psycopg2,How to connect to AWS EC2 and AWS RDS using my company's remote access VPN?,AWS RDS with SSIS ODBC,AWS + RDS result backend celery,AWS RDS: How to use AD Account to Authenticate,How to configure sequalize & node mysql to use new aws rds root cert rds-ca-2019,How to Use Amazon AWS Credentials to Access/Query Tables in AWS RDS Instance through Mobile App,AWS RDS Postgres maximum instance and table size,AWS RDS (MySQL) Secure Connection,How to use Terraform to disable deletion protection on AWS RDS?,"How to create a Database Replica in AWS RDS using RDS instance, created using DatabaseInstanceFromSnapshot",How to use AWS DMS from a region to an other?,How to develop a AWS Web App that uses AWS RDS locally?,How to connect to AWS RDS using Golang?,How to use pg_restore with AWS RDS correctly to restore postgresql database
question,\nWe want to use AWS RDS Read replica by psyco...,\nMy company's AWS security inbound rules are ...,\nHow would you pull in data from an ODBC sour...,\nHow can I write a valid URL to use celery re...,\nI have an AWS RDS Instance and AWS Directory...,\nAs AWS changes their root ssl cert for rds s...,\nI've been quite confused for a couple of wee...,\nThanks in advance!\nI am planning to use AWS...,\nI have following aws resources an EC2 Instan...,\nI used Terraform to bring up an AWS RDS SQL ...,\nI want to create a DB Read Replica using aws...,\nI am trying to use AWS DMS to move data from...,"\nBefore moving to Amazon Web Services, I was ...",\nI am trying to connect to the AWS RDS using ...,\nI am trying to restore my Postgresql databas...
answer,[],[\nThere is a few moving parts here so stay wi...,[\ndid you figure out?\nOne thing I would say ...,"[\nTo use Celery with AWS RDS as backend, you ...",[],[\nEnsure you update to the latest node mysql ...,[\n\nI know I have to use some kind of HTTP/RE...,"[\nIn addition to RDS for PostgreSQL, which ha...","[\nRDS server doesn't served SSH. \nHowever, y...","[\nThis would be two distinct API invocations,...",[],"[\nAWS provides this whitepaper called ""Migrat...",[\nAWS is deny be default so you cannot access...,[],"[\nOk, since this is apparently useful to some..."
url,https://stackoverflow.com/questions/47883071/h...,https://superuser.com/questions/1533843/how-to...,https://stackoverflow.com/questions/64135057/a...,https://stackoverflow.com/questions/65896132/a...,https://stackoverflow.com/questions/74283498/a...,https://stackoverflow.com/questions/59348048/h...,https://stackoverflow.com/questions/34302667/h...,https://stackoverflow.com/questions/54626070/a...,https://stackoverflow.com/questions/37032050/a...,https://stackoverflow.com/questions/69412371/h...,https://stackoverflow.com/questions/73646196/h...,https://stackoverflow.com/questions/52151009/h...,https://stackoverflow.com/questions/58885959/h...,https://stackoverflow.com/questions/75431876/h...,https://stackoverflow.com/questions/56462616/h...


In [16]:
a = content['https://stackoverflow.com/questions/55769540/position-sticky-not-working-no-matter-what']
a.pop('question_title')
a

{'question': '\nTrying to get a class to remain sticky and constantly be in view when the user scrolls down the page, but doesn\'t seem to work no matter what. \nI tired: \n.entry-summary{\nposition: -webkit-sticky; /* Safari */\n  position: sticky;\n  top: 0;}\n\nBut this does nothing. \nThe full code can be seen below and is part of this webpage https://aqwaa.net/product/constriction/: \n<div class="qode-single-product-summary">\n    <div class="summary entry-summary">\n        <div class="clearfix">\n            <h1 class="product_title entry-title">Constriction</h1><p class="price"><span class="woocs_price_code" data-product-id="459"><span class="woocommerce-Price-amount amount"><span class="woocommerce-Price-currencySymbol">€</span>173.39</span></span></p>\n<div class="woocommerce-product-details__short-description">\n    <p><strong>Mental Health Description:&nbsp;</strong>So often a symptom of anxiety is the feeling of a tightness or tension in the rib cage area. Anxiety has the 

In [50]:
soup = BeautifulSoup(content, parser='html.parser')
links = soup.find_all(class_='result-link')
for link in links:
    print(link.a['href'])
a_links = [link.a['href'] for link in a]
# print(len(links))


https://stackoverflow.com/questions/55769540/position-sticky-not-working-no-matter-what
https://math.stackexchange.com/questions/3665900/determining-whether-a-matrix-is-positive-definite-from-its-lu-decomposition
https://electronics.stackexchange.com/questions/499491/splicing-ac-adaptor-cord-which-is-positive-and-which-is-negative
https://electronics.stackexchange.com/questions/227190/why-doesnt-positive-negative-terminal-matter-in-some-electrical-applicances
https://stackoverflow.com/questions/7129597/how-to-keep-track-of-a-position-no-matter-what-screen-size-is
https://stackoverflow.com/questions/74447652/if-the-value-in-sql-is-positive-how-can-i-do-this-operation-if-it-is-negative
https://stackoverflow.com/questions/69641323/why-codeblocks-asking-me-to-add-a-after-elses-condition-if-i-add-a-th
https://worldbuilding.stackexchange.com/questions/101581/negative-energy-beam-negative-matter-projectiles
https://mathematica.stackexchange.com/questions/28659/manipulate-doesnt-work-for-plott

In [120]:

soup = BeautifulSoup(content, parser='html.parser')
#links = content.text
links = soup.find_all('a')
herfs = []
for link in links:
    herf = link.get('herf')
    herfs.append(herf)

TypeError: object of type 'WebElement' has no len()

In [71]:
soup = BeautifulSoup(parser='html.parser', content=result)
soup.find_all('herf')

TypeError: __init__() got an unexpected keyword argument 'content'

In [ ]:
url = 'https://math.stackexchange.com/questions/3665900/determining-whether-a-matrix-is-positive-definite-from-its-lu-decomposition'
res = requests.get(url)
soup = BeautifulSoup(res.text, parser='html_parser')
post_body = soup.find_all('div', class_='s-prose js-post-body',itemprop='text')
question_title = soup.find('h1',class_='fs-headline1 ow-break-word mb8 flex--item fl1',itemprop='name')

answer_body = [i.text for i in post_body if i.text]
answer_body

In [ ]:
answer_body[1:len(answer_body)]

In [ ]:
a =[]
for i in content:
    if i.text:
        a.append(i.text)
len(a)